<span STYLE="font-size:200%"> 
    Lineplot showing mean values of each group
</span>

Docker image: gnasello/datascience-env:2023-01-23\
Latest update: 06 February 2024

# Load required packages

In [ ]:
library(ggplot2)
library(dataprepUtils)
library(ggplotUtils)
library(statsUtils)
library(rstatix)
library(lme4)
library(emmeans)

# Load data

In [ ]:
filetable <- 'data.csv'

title <- 'Plot Title'
xlabel <- 'Time (day)'
ylabel <- 'Cumulative release (%)'

# color of each group
#scale_color_manual.values <- c("#8b8c8cff", "#4dbbd5ff", "#ff6666ff", "#4d9999ff")
scale_color_manual.values <- c("#dca01cff", "#386e28ff", "#004488", "#4dbbd5ff")

# scale_color_manual.values <- c("Blank" = "#8b8c8cff", 
#                                "αCDs" = "#4dbbd5ff", 
#                                "AS" = "#ff6666ff", 
#                                "αCDs + AS" = "#4d9999ff")

# Use the function to read and process the data
df <- read_and_process_data(filetable, 
                            x_col = "day", 
                            y_col = 'cumulative_percentage',
                            group = 'condition',
                            # xlabels_ordered = names(scale_color_manual.values),
                           )
head(df)
tail(df)

# Line plot of `y` over `x`

## Summarize the data

The function below will be used to calculate the mean and the standard deviation, for the variable of interest, in each group. See [tutorial](http://www.sthda.com/english/wiki/ggplot2-line-plot-quick-start-guide-r-software-and-data-visualization#line-graph-with-error-bars)

In [ ]:
df_summary <- data_summary(df, varname="y", 
                           groupnames=c("group", "x"))
df_summary <- df_summary[complete.cases(df_summary$x),]

df_summary

## Bar plot with point data

In [ ]:
color='#404040ff'

ylim <- c(0,110)
xlim <- c(0,8.5)
width=9
height=7.6

p <- ggplot() +
     geom_line(data=df_summary, aes(x=x, y=mean, color=group), linewidth=1.5) +
     geom_point(data=df_summary, aes(x=x, y=mean, color=group), size=4.5, shape=15, stroke=2) +
     geom_errorbar(data=df_summary, aes(x=x, y=mean, ymin=mean-sd, ymax=mean+sd), width=.4, linewidth=0.75, color=color)

img <- apply_minimal_theme(p, width=width, height=height,
                           title=title,
                           plot.title = element_text(size = 24),
                           axis.text=element_text(size=24),
                           axis.title=element_text(size=26),
                           xlabel = xlabel, 
                           ylabel= ylabel, 
                           xlim = xlim,
                           ylim = ylim,
                           scale_color_manual.values = scale_color_manual.values,
                          ) + 
            theme(axis.line.x.bottom=element_line(linewidth=0.75),
                  axis.line.y.left=element_line(linewidth=0.75))#+
            # scale_x_continuous(breaks = seq(0, 16, by = 3))

img

# Export plot

In [ ]:
filename <- tools::file_path_sans_ext(filetable)
filename

## `.png` file

In [ ]:
fileoutput <- paste(filename,'.png', sep='')
fileoutput

In [ ]:
ggsave(file=fileoutput, plot=img, width=width, height=height, bg='white')

## `.svg` file

In [ ]:
fileoutput <- paste(filename,'.svg', sep='')
fileoutput

In [ ]:
ggsave(file=fileoutput, plot=img, width=width, height=height)

# Two-way mixed ANOVA

Read more on the theory of this method on [Datanovia](https://www.datanovia.com/en/lessons/mixed-anova-in-r/#two-way-mixed)

## Computation

In [ ]:
# Two-way mixed ANOVA test
res.aov <- anova_test(
                      data = df, dv = y, wid = sample,
                      between = group, within = x
                      )
get_anova_table(res.aov)

## Post-hoc tests

### Compute simple main effect

Procedure for a significant two-way interaction

In [ ]:
# Effect of group at each time point
one.way <- df %>%
              group_by(x) %>%
              anova_test(dv = y, wid = sample, between = group) %>%
              get_anova_table() %>%
              adjust_pvalue(method = "bonferroni")
one.way

Procedure for non-significant two-way interaction

In [ ]:
df %>%
      pairwise_t_test(
        y ~ group, paired = TRUE, 
        p.adjust.method = "bonferroni"
      )

### Simple pairwise comparisons

In [ ]:
# Pairwise comparisons between group levels
pwc <- df %>%
          group_by(x) %>%
          pairwise_t_test(y ~ group, p.adjust.method = "bonferroni")
pwc